In [0]:
a = 1

In [0]:
a

1

In [1]:
!wget https://s3.amazonaws.com/conceptnet/downloads/2018/edges/conceptnet-assertions-5.6.0.csv.gz

--2019-07-09 05:36:29--  https://s3.amazonaws.com/conceptnet/downloads/2018/edges/conceptnet-assertions-5.6.0.csv.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.139.221
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.139.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 471053810 (449M) [text/csv]
Saving to: ‘conceptnet-assertions-5.6.0.csv.gz’

conceptnet-assertio 100%[===================>] 449.23M  46.7MB/s    in 9.3s    

2019-07-09 05:36:39 (48.3 MB/s) - ‘conceptnet-assertions-5.6.0.csv.gz’ saved [471053810/471053810]



In [0]:
!gunzip conceptnet-assertions-5.6.0.csv.gz

In [0]:
graph = {}
i = 1

with open("conceptnet-assertions-5.6.0.csv","r") as f1:
	for x in f1:
		# print(i)
		i = i +1
		a,b,c,d,e = x.split('	',4)


		# print(a)
		# print(b)
		# print(c)
		# print(d)
		# print(e)


		# relation
		b1,b2,b3 = b.split('/',2)

		if("/" in b3):
			b33,b4 = b3.split('/')
		else:
			b4 = b3	
 		# print(b1)
		# print(b2)
		# print(b3)
		# print(b4)


		# entity1
		c1,c2,c3,c4 = c.split('/',3)

		if("/" in c4):
			c4,c5 = c4.split('/')
		
		# print(c1)
		# print(c2)
		# print(c3)
		# print(c4)
		# print(c5)

		# entity2
		d1,d2,d3,d4 = d.split('/',3)

		if("/" in d4):
			d4,d5 = d4.split('/',1)
		# print(d1)
		# print(d2)
		# print(d3)
		# print(d4)


		# print(c4)
		# print(b4)
		# print(d4)

		c4 = c4.rstrip()
		b4 = b4.rstrip()
		d4 = d4.rstrip()
		if(c4 not in graph):
			graph[c4] = []
			
		graph[c4].append((b4,d4))

--2019-06-27 10:36:13--  https://github.com/dongwookim-ml/kg-data/blob/master/nation/triples.txt
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘triples.txt’

triples.txt             [  <=>               ] 440.75K  1.47MB/s    in 0.3s    

2019-06-27 10:36:14 (1.47 MB/s) - ‘triples.txt’ saved [451330]



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
s_entities = {}
s_rel = {}
sparse_graph = {}

i = 0
with open("entity2id.txt","r") as f1:
	for x in f1:
		if(i != 0):
			a,b = x.split(' ')
			a = a.rstrip()
			b = b.rstrip()
			s_entities[b] = a
		
		i = i + 1	

i = 0

with open("relation2id.txt","r") as f1:
	for x in f1:
		if(i != 0):
			a,b = x.split(' ')
			a = a.rstrip()
			b = b.rstrip()
			s_rel[b] = a
		
		i = i + 1	

i = 0

ent_deg = {}

with open("triples.txt","r") as f1:
	for x in f1:
		
		r,e1,e2 = x.split('	')
		
		r = r.rstrip()
		e1 = e1.rstrip()
		e2 = e2.rstrip()

		if(s_entities[e1] not in sparse_graph):
			sparse_graph[ s_entities[e1] ] = []
			ent_deg[ s_entities[e1] ] = 0
            
		sparse_graph[ s_entities[e1] ].append( ( s_rel[r] , s_entities[e2] ) )
		ent_deg[ s_entities[e1] ] =  ent_deg[s_entities[e1]] + 1 


In [31]:
ent_sot = sorted(ent_deg.items(), key=lambda x: x[1] , reverse  = True)
cnt = 0
num = 0
for (ent,n) in ent_sot:
	
	print(ent)
	if(cnt > 7):
		break
	cnt = cnt + 1

	if(ent in graph):
		for r in graph[ent]:

			if(r not in sparse_graph[ent]):
				sparse_graph[ent].append(r)

				num = num + 1
        
        
				if(r[0] not in s_rel.values()):
					s_rel[len(s_rel)] = r[0].rstrip() 

				if(r[1] not in s_entities.values()):
					s_entities[ len(s_entities) ] = r[1].rstrip() 

usa
uk
ussr
poland
egypt
china
netherlands
india
cuba


In [32]:
num

1074

In [0]:
import networkx as nx
G=nx.Graph()

In [34]:
len(s_entities)

975

In [0]:
ls = []

for i in range(0,len(s_entities)):
  ls.append(i)

In [0]:
 G.add_nodes_from(ls)

In [0]:
map_entities = {}
map_rel = {}

with open("Entity2id.txt","w+") as ent2id:
	ent2id.write(str(len(s_entities)) + "\n")
	for ent in s_entities:
		ent2id.write(str(s_entities[ent]) + " " + str(ent) + "\n")
		map_entities[str(s_entities[ent])] = str(ent)
		i = i + 1

i = 0



with open("Relation2id.txt","w+") as rel2id:
	rel2id.write(str(len(s_rel)) + "\n")
	for rel in s_rel:
		rel2id.write(str(s_rel[rel]) + " " + str(rel) + "\n")
		map_rel[str(s_rel[rel])] = str(rel)
		i = i + 1


l = 0
with open("Triples.txt","w+") as t2id:
	for ent in sparse_graph:
		for rel in sparse_graph[ent]:
			t2id.write(str(map_entities[ent]) + " " +str(map_entities[rel[1].rstrip()]) + " " + str(map_rel[rel[0].rstrip()]) + "\n")
			G.add_edge( int(map_entities[ent]) , int(map_entities[rel[1].rstrip()]) )
			l = l + 1


In [28]:
r=nx.degree_assortativity_coefficient(G,'in','in')
print("%3.1f"%r)

-0.7


In [43]:
r=nx.degree_assortativity_coefficient(G,'out','in')
print(r)

-0.6738927293633622


In [44]:
r=nx.degree_assortativity_coefficient(G,'in','in')
print(r)

-0.6738927293633622


In [45]:
r=nx.degree_assortativity_coefficient(G,'out','out')
print(r)

-0.6738927293633622
